## Model Design Rationale (EfficientNetB0)

This model was trained on a small ultrasound dataset (<800 images) for 3-way classification: benign, malignant, and normal.

### Why EfficientNetB0?
Although we initially tested larger backbones like EfficientNetB3, we found that EfficientNetB0 consistently outperformed it on this limited dataset. B0's smaller size makes it less prone to overfitting and easier to optimize. It provides strong performance with fewer parameters, faster training, and greater stability—making it a better match for small-scale medical imaging tasks. We briefly tested alternatives like ResNet50 and MobileNetV2, but they didn't perform quite as well. EfficientNetB0 provides a lightweight model, that is very fast and accurate for its size with strong pretrained features.

### Why Freeze Most of the Base?
While full fine-tuning led to marginal gains, the most stable improvements came from a 2-phase approach: training only the classification head first, then fine-tuning just the final convolutional block (with BatchNorm layers frozen). This balances generalization and overfitting risk on limited data, leveraging EfficientNetB0's pretrained features while still adapting the top layers.

### Augmentation Strategy
We applied geometric and brightness augmentations (rotation, flips, zoom, etc.) to increase diversity without harming structural integrity. Cutout and mixup were avoided after brief tests showed performance drops. Each epoch applies a random type of augmentation and all images are trained on once per epoch, ensuring that over many epochs the model sees varied versions of each image to enhance generalization without increasing dataset size.

### Class Imbalance Handling
The dataset had class imbalances, with some classes (e.g., benign) appearing more frequently than others. To correct this, we used `class_weight='balanced'` to adjust for skewed class distributions. This helped prevent the model from overpredicting common classes (e.g., benign).

In [ ]:
# Environment Versions Used:
# TensorFlow:         2.1.0
# Keras:              1.1.0
# Pandas:             1.1.3
# Numpy:              1.18.5
# Scikit-Learn:       0.22.2
# Matplotlib:         3.1.3

"""
MRI Classification - EfficientNetB0 (3-class: benign, malignant, normal)

This model trains a classifier head on a frozen EfficientNetB0 base.
Key strategies:
- Uses 5-fold stratified cross-validation
- Applies geometric & brightness augmentations
- Balances class imbalance with class weights
- Avoids fine-tuning the base due to instability on small data
"""

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from efficientnet.tfkeras import EfficientNetB0
from tensorflow.keras.models import Model, clone_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.callbacks import ReduceLROnPlateau
import os
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt # For plotting history
import os
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import gc

# --- Configuration ---
data_dir = 'data'
all_data_dir = os.path.join(data_dir, 'all')

if not os.path.exists(all_data_dir):
    raise FileNotFoundError(f"Combined data directory not found: {all_data_dir}")

img_height, img_width = 224, 224
batch_size = 16
num_classes = 3
max_epochs_per_fold = 75 # Keep max epochs high, let early stopping work
n_splits = 5

# --- 1. Load All Data Information ---
print("Loading all image file paths and labels...")
filepaths = []
labels = []
# Use list comprehension for potentially cleaner directory filtering
class_names = sorted([d for d in os.listdir(all_data_dir) if os.path.isdir(os.path.join(all_data_dir, d))])
if len(class_names) != num_classes:
    print(f"Warning: Found {len(class_names)} directories/classes, expected {num_classes}. Adjusting.")
    num_classes = len(class_names)

class_map = {name: i for i, name in enumerate(class_names)}

for class_name in class_names:
    class_dir = os.path.join(all_data_dir, class_name)
    if os.path.isdir(class_dir):
        for fname in os.listdir(class_dir):
            fpath = os.path.join(class_dir, fname)
            # Basic check for image files
            if os.path.isfile(fpath) and fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                filepaths.append(os.path.normpath(fpath))
                labels.append(class_name)

all_data_df = pd.DataFrame({'filepath': filepaths, 'label': labels})
all_data_df.dropna(subset=['filepath'], inplace=True) # Ensure no missing paths

print(f"Loaded {len(all_data_df)} valid image paths.")
if len(all_data_df) == 0:
     raise ValueError("No valid image files found. Check 'all_data_dir' and image file extensions.")
print("Class distribution in full dataset:")
print(all_data_df['label'].value_counts())

# --- 2. Data Augmentation Generators ---
# Back to geometric + brightness augmentation (NO Cutout)
print("Setting up Data Generators (Geometric + Brightness)...")
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
    # No preprocessing_function needed now
)

val_datagen = ImageDataGenerator(rescale=1./255)

# --- 3. K-Fold Cross-Validation Setup ---
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
fold_results = []

# --- Function to build the model (SIMPLIFIED HEAD) ---
def build_model(input_shape, num_classes):
    print("Building fresh model instance with SIMPLIFIED head...")
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False # Keep frozen

    # *** Simplified Head ***
    x = base_model.output
    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dropout(0.5)(x) # Single dropout after GAP
    predictions = Dense(num_classes, activation='softmax', name='predictions')(x)
    # Removed intermediate Dense(256) and second Dropout

    model = Model(inputs=base_model.input, outputs=predictions)

    # Keep LR and optimizer same as previous successful run
    initial_learning_rate = 3e-4
    optimizer = Adam(learning_rate=initial_learning_rate)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    print("Model compiled.")
    return model

# --- 4. K-Fold Training Loop ---
print(f"\n--- Starting {n_splits}-Fold Cross-Validation with Simplified Head ---")
for fold, (train_idx, val_idx) in enumerate(skf.split(all_data_df['filepath'], all_data_df['label'])):
    print(f"\n===== FOLD {fold+1}/{n_splits} =====")

    train_df = all_data_df.iloc[train_idx]
    val_df = all_data_df.iloc[val_idx]
    print(f"Fold {fold+1}: Train size={len(train_df)}, Val size={len(val_df)}")

    train_labels_int = train_df['label'].map(class_map).values
    unique_train_labels = np.unique(train_labels_int)
    try:
        fold_class_weights_array = compute_class_weight(
            class_weight='balanced',
            classes=unique_train_labels,
            y=train_labels_int
        )
        fold_class_weights = {label_int: 0.0 for label_int in range(num_classes)}
        for label_int, weight in zip(unique_train_labels, fold_class_weights_array):
            fold_class_weights[label_int] = weight
        if any(w == 0.0 for w in fold_class_weights.values()) and len(unique_train_labels) < num_classes:
             print(f"Warning: Fold {fold+1} training set missing some classes. Weights: {fold_class_weights}")
    except ValueError as e:
        print(f"Warning: Could not compute class weights for Fold {fold+1}. Error: {e}. Setting weights to None.")
        fold_class_weights = None

    print(f"Fold {fold+1} Class Weights: {fold_class_weights}")

    fold_train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col='filepath',
        y_col='label',
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        classes=class_names,
        shuffle=True
    )

    fold_val_generator = val_datagen.flow_from_dataframe(
        dataframe=val_df,
        x_col='filepath',
        y_col='label',
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        classes=class_names,
        shuffle=False
    )

    model = build_model((img_height, img_width, 3), num_classes)

    # Callbacks (same as previous successful run)
    early_stopping = EarlyStopping(monitor='val_loss', patience=15, verbose=1, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, verbose=1, min_lr=1e-7)
    callbacks = [early_stopping, reduce_lr]

    print(f"--- Training Fold {fold+1} ---")
    history = model.fit(
        fold_train_generator,
        epochs=max_epochs_per_fold,
        validation_data=fold_val_generator,
        callbacks=callbacks,
        class_weight=fold_class_weights,
        verbose=1
    )

    print(f"--- Evaluating Fold {fold+1} ---")
    loss, accuracy = model.evaluate(fold_val_generator, verbose=0)
    print(f"Fold {fold+1} Validation Loss: {loss:.4f}")
    print(f"Fold {fold+1} Validation Accuracy: {accuracy:.4f}")
    fold_results.append(accuracy)

    # Clean up memory
    del model
    del fold_train_generator
    del fold_val_generator
    tf.keras.backend.clear_session()
    gc.collect()

# --- 5. Report Final Cross-Validation Results ---
print("\n--- Cross-Validation Results with Simplified Head ---")
mean_accuracy = np.mean(fold_results)
std_accuracy = np.std(fold_results)
print(f"Fold Validation Accuracies: {[f'{acc:.4f}' for acc in fold_results]}")
print(f"Average Validation Accuracy: {mean_accuracy:.4f}")
print(f"Standard Deviation of Validation Accuracy: {std_accuracy:.4f}")


In [ ]:
MODEL_NAME = "efficientnet"
model_path = f"{MODEL_NAME}.h5"

print("\n--- Training Final Model on ALL Data ---")

# --- Configuration (repeat relevant parts) ---
data_dir = 'data'
all_data_dir = os.path.join(data_dir, 'all')
img_height, img_width = 224, 224
batch_size = 16
num_classes = 3 # Adjust if different
# Determine training epochs based on K-Fold results (where val_loss plateaued/stopped)
final_epochs = 45 # <<< Adjust this number as needed (e.g., 40-50 seems reasonable)

# --- 1. Load All Data Info (same as K-Fold start) ---
print("Loading all image file paths and labels for final training...")
filepaths = []
labels = []
class_names = sorted([d for d in os.listdir(all_data_dir) if os.path.isdir(os.path.join(all_data_dir, d))])
if len(class_names) != num_classes:
    print(f"Warning: Found {len(class_names)} classes. Adjusting num_classes.")
    num_classes = len(class_names)
class_map = {name: i for i, name in enumerate(class_names)}
for class_name in class_names:
    class_dir = os.path.join(all_data_dir, class_name)
    if os.path.isdir(class_dir):
        for fname in os.listdir(class_dir):
            fpath = os.path.join(class_dir, fname)
            if os.path.isfile(fpath) and fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                filepaths.append(os.path.normpath(fpath))
                labels.append(class_name)
all_data_df = pd.DataFrame({'filepath': filepaths, 'label': labels})
all_data_df.dropna(subset=['filepath'], inplace=True)
print(f"Loaded {len(all_data_df)} valid image paths for final training.")
if len(all_data_df) == 0:
     raise ValueError("No valid image files found for final training.")

# --- 2. Data Generator for Final Training (No Cutout) ---
print("Setting up Data Generator for final training (Geometric + Brightness)...")
final_train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
    # No preprocessing_function
)
final_train_generator = final_train_datagen.flow_from_dataframe(
    dataframe=all_data_df,
    x_col='filepath',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    classes=class_names,
    shuffle=True
)

# --- 3. Calculate Class Weights for ALL Data ---
print("Calculating class weights for the full dataset...")
all_labels_int = all_data_df['label'].map(class_map).values
all_class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.array(list(class_map.values())),
    y=all_labels_int
)
all_class_weights = {label_int: weight for label_int, weight in zip(class_map.values(), all_class_weights_array)}
print(f"Full Dataset Class Weights: {all_class_weights}")


# --- 4. Build and Compile Final Model (Simplified Head) ---
# Function to build the specific model configuration
def build_final_model(input_shape, num_classes):
    print("Building final model instance with SIMPLIFIED head...")
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False # Keep frozen

    x = base_model.output
    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dropout(0.5)(x) # Single dropout after GAP
    predictions = Dense(num_classes, activation='softmax', name='predictions')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model # Return uncompiled model

final_model = build_final_model((img_height, img_width, 3), num_classes)

# Compile the model
initial_learning_rate = 3e-4 # Start with the LR used in K-Fold
optimizer = Adam(learning_rate=initial_learning_rate)
final_model.compile(optimizer=optimizer,
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
print("Final model compiled.")
final_model.summary()

# --- 5. Define Callback for Final Training ---
reduce_lr_final = ReduceLROnPlateau(
    monitor='loss', # Monitor training loss
    factor=0.2,
    patience=7,
    verbose=1,
    min_lr=1e-7
    )
callbacks_final = [reduce_lr_final]

# --- 6. Train the Final Model ---
print(f"\n--- Starting Final Training on ALL Data for {final_epochs} epochs ---")
history_final = final_model.fit(
    final_train_generator,
    epochs=final_epochs,
    callbacks=callbacks_final,
    class_weight=all_class_weights,
    verbose=1 # Show progress
)

# --- 7. Save the Final Model ---
final_model.save(model_path)
print(f"\n--- Final Model Saved to {model_path} ---")

# --- 8. Optional: Plot Final Training History ---
try:
    def plot_final_history(history):
        acc = history.history.get('accuracy', [])
        loss = history.history.get('loss', [])

        if not acc:
             print("No training history to plot.")
             return

        epochs_range = range(len(acc))
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range, acc, label='Training Accuracy')
        plt.title('Final Training Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend(loc='lower right'); plt.grid(True)
        plt.ylim([min(0.0, min(acc)*0.9), 1.0])
        plt.subplot(1, 2, 2)
        plt.plot(epochs_range, loss, label='Training Loss')
        plt.title('Final Training Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend(loc='upper right'); plt.grid(True)
        plt.ylim([0, max(loss)*1.1])
        plt.tight_layout()
        plt.show()

    print("\nPlotting final training history...")
    plot_final_history(history_final)
except Exception as e:
    print(f"\nError plotting history: {e}")

Example to show the models output on a test image

In [ ]:
img_path = "mal_test.png"
img_height, img_width = 300, 300
class_names = ['benign', 'malignant', 'normal'] 

# --- Load Model ---
model = load_model(model_path)
print(f"✅ Loaded model from: {model_path}")

# --- Load and Preprocess Image ---
img = image.load_img(img_path, target_size=(img_height, img_width))
img_array = image.img_to_array(img) / 255.0
img_batch = np.expand_dims(img_array, axis=0)

# --- Predict ---
pred_probs = model.predict(img_batch)[0]
pred_class_idx = np.argmax(pred_probs)
pred_class_name = class_names[pred_class_idx]
confidence = pred_probs[pred_class_idx]

# --- Display Image & Prediction ---
plt.figure(figsize=(6, 4))
plt.imshow(img_array)
plt.axis('off')
plt.title(f"Predicted: {pred_class_name} ({confidence*100:.1f}%)", fontsize=14)
plt.show()

# --- Display Confidence for All Classes ---
plt.figure(figsize=(7, 4))
bars = plt.bar(class_names, pred_probs * 100)
for i, bar in enumerate(bars):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 1,
             f"{pred_probs[i]*100:.1f}%", ha='center', fontsize=12)
plt.ylim(0, 100)
plt.ylabel("Confidence (%)")
plt.title("Model Confidence per Category")
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()